#### This file is used to train the more robust attack model using the adversarial training.

##### Generate the adversarial examples using the PGD method.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
from art.attacks.evasion import DeepFool
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("mnist"))
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:500], y_test[:500]
im_shape = x_train[0].shape

#The structure of the attack model.
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
attack_model = model

classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)
classifier.save("./adv_data/models/natural_attack_model.h5")

In [ ]:
adv_crafter_pgd = ProjectedGradientDescent(estimator=classifier, norm=np.inf, eps=0.3, eps_step=0.01, batch_size=32)
x_train_pgdi_adv_attack = adv_crafter_pgd.generate(x = x_train)
x_train_ = np.append(x_train, x_train_pgdi_adv_attack, axis = 0)
y_train_ = np.append(y_train, y_train, axis = 0)

##### Training the more robust attack model using the adversarial examples generated by the PGD.

In [ ]:
path_0 = "./adv_data/test_evasion/"
attack_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
classifier = KerasClassifier(model=attack_model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)
classifier.save("./adv_data/models/mnist" + "_" + "pgdi" + "_" + "0.3" + ".h5")

##### Evaluating the more robust attack model using the adversarial examples.

In [ ]:
from keras.models import load_model 
#Loading the model, including the natural attack model and the more robust attack model.
classifier_nat = load_model("./adv_data/models/natural_attack_model.h5")
classifier_pgdi = load_model("./adv_data/models/mnist" + "_" + "pgdi" + "_" + "0.3" + ".h5")
#Loading the adversarial examples.
x_test_adv_cw2 = np.load("./adv_data/x_adv/x_test_adv_cw2.npy")
x_test_adv_fgsm = np.load("./adv_data/x_adv/x_test_adv_fgsm.npy")
x_test_adv_jsma = np.load("./adv_data/x_adv/x_test_adv_jsma.npy")
x_test_adv_pgd = np.load("./adv_data/x_adv/x_test_adv_pgd.npy")
#Defining the evaluation function.
def eval_on_model(classifier, x_test_adv, y_test):
    preds = np.argmax(classifier.predict(x_test_adv), axis=1)
    acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
    #print("test_pgd %.2f%%",(acc*100))
    return acc
#Evaluating the adversarial example on the natural attack model.
acc_cw2 = eval_on_model(classifier_nat, x_test_adv_cw2, y_test)
acc_jsma = eval_on_model(classifier_nat, x_test_adv_jsma, y_test)
acc_fgsm = eval_on_model(classifier_nat, x_test_adv_fgsm, y_test)
acc_pgd = eval_on_model(classifier_nat, x_test_adv_pgd, y_test)
#Print the result
print(acc_cw2)
print(acc_jsma)
print(acc_fgsm)
print(acc_pgd)

In [ ]:
#Evaluating the adversarial example on the more robust attack model.
acc_cw2 = eval_on_model(classifier_pgdi, x_test_adv_cw2, y_test)
acc_jsma = eval_on_model(classifier_pgdi, x_test_adv_jsma, y_test)
acc_fgsm = eval_on_model(classifier_pgdi, x_test_adv_fgsm, y_test)
acc_pgd = eval_on_model(classifier_pgdi, x_test_adv_pgd, y_test)
#Print the result
print(acc_cw2)
print(acc_jsma)
print(acc_fgsm)
print(acc_pgd)